# 🤖 AI 챗봇 멘토링 - 1차시: 기본 챗봇 구현

## 🎯 학습 목표
- OpenAI API 연동 방법 이해
- 스트리밍 응답 구현
- 토큰 사용량 모니터링
- API 키 로테이션 로직
- 실무용 로깅 시스템

## 📋 사전 준비사항
1. OpenAI API 키 발급
2. Python 3.8+ 설치
3. 필요한 라이브러리 설치 (`pip install -r requirements.txt`)
4. `.env` 파일 설정

## 1️⃣ 환경 설정 및 라이브러리 임포트

In [ ]:
#!/usr/bin/env python3
import os
import time
import json
import logging
from typing import Iterator, Dict, Any, Optional, List
from datetime import datetime, timedelta
from dataclasses import dataclass, asdict
from functools import wraps
from openai import OpenAI
import redis

# 환경변수 로드 (개발용)
from dotenv import load_dotenv
load_dotenv()

print("✅ 라이브러리 임포트 완료")
print(f"📍 현재 작업 디렉토리: {os.getcwd()}")
print(f"🔑 API 키 설정 상태: {'✅ 설정됨' if os.getenv('OPENAI_API_KEY') else '❌ 미설정'}")

## 2️⃣ 로깅 시스템 설정

실무에서는 **상세한 로깅**이 필수입니다. API 호출, 토큰 사용량, 에러 상황을 모두 추적해야 합니다.

In [ ]:
# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),  # 콘솔 출력
        logging.FileHandler('chatbot_lesson1.log', encoding='utf-8')  # 파일 저장
    ]
)
logger = logging.getLogger(__name__)

def log_function_call(func):
    """함수 호출을 로깅하는 데코레이터"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        logger.debug(f"[ENTER] {func.__name__}", extra={
            "function": func.__name__, 
            "args_count": len(args),
            "kwargs_keys": list(kwargs.keys())
        })
        
        try:
            result = func(*args, **kwargs)
            elapsed = time.time() - start_time
            logger.debug(f"[EXIT] {func.__name__} - SUCCESS ({elapsed:.3f}s)")
            return result
        except Exception as e:
            elapsed = time.time() - start_time
            logger.error(f"[EXIT] {func.__name__} - ERROR: {str(e)} ({elapsed:.3f}s)")
            raise
    return wrapper

print("✅ 로깅 시스템 설정 완료")
logger.info("Jupyter Notebook 실습 시작")

## 3️⃣ 기본 설정 및 데이터 모델

실무에서는 **타입 힌트**와 **데이터 클래스**를 활용해 코드의 안정성을 높입니다.

In [ ]:
@dataclass
class ChatMessage:
    """채팅 메시지 구조"""
    role: str  # "user" or "assistant" or "system"
    content: str
    timestamp: datetime
    tokens_used: int = 0
    processing_time: float = 0.0
    model: str = ""

# 기본 설정값
DEFAULT_CONFIG = {
    "model": os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    "max_tokens": int(os.getenv("MAX_TOKENS", "500")),
    "temperature": float(os.getenv("TEMPERATURE", "0.7")),
    "api_key": os.getenv("OPENAI_API_KEY"),
    "daily_token_limit": int(os.getenv("DAILY_TOKEN_LIMIT", "10000"))
}

# 설정 출력
print("⚙️ 현재 설정:")
for key, value in DEFAULT_CONFIG.items():
    if key == "api_key":
        display_value = f"{value[:8]}...{value[-4:]}" if value else "❌ 미설정"
    else:
        display_value = value
    print(f"  {key}: {display_value}")

# API 키 검증
if not DEFAULT_CONFIG["api_key"]:
    print("\n❌ OpenAI API 키가 설정되지 않았습니다!")
    print("   다음 중 하나를 실행하세요:")
    print("   1. .env 파일에 OPENAI_API_KEY 추가")
    print("   2. 환경변수 설정: export OPENAI_API_KEY='your-key'")
    
    # 노트북에서 직접 입력 (개발용)
    api_key_input = input("\n🔑 API 키를 직접 입력하세요 (선택사항): ")
    if api_key_input.strip():
        os.environ["OPENAI_API_KEY"] = api_key_input.strip()
        DEFAULT_CONFIG["api_key"] = api_key_input.strip()
        print("✅ API 키가 설정되었습니다.")
else:
    print("✅ API 키 설정 확인")

## 4️⃣ 토큰 사용량 모니터링 클래스

실무에서는 **비용 관리**가 중요합니다. 토큰 사용량을 추적하고 제한을 설정해야 합니다.

In [ ]:
class TokenMonitor:
    """토큰 사용량 모니터링 및 비용 계산"""
    
    # 2024년 8월 기준 모델별 토큰당 가격 (USD)
    TOKEN_PRICES = {
        "gpt-4o-mini": {"input": 0.00015 / 1000, "output": 0.0006 / 1000},
        "gpt-4o": {"input": 0.005 / 1000, "output": 0.015 / 1000},
        "gpt-3.5-turbo": {"input": 0.001 / 1000, "output": 0.002 / 1000},
    }
    
    def __init__(self):
        self.daily_usage = {}  # 사용자별 일일 사용량
        self.session_stats = {
            "total_requests": 0,
            "total_tokens": 0,
            "total_cost": 0.0,
            "start_time": datetime.now()
        }
        logger.info("토큰 모니터 초기화 완료")
    
    def calculate_cost(self, model: str, input_tokens: int, output_tokens: int) -> float:
        """토큰 사용량 기반 비용 계산"""
        if model not in self.TOKEN_PRICES:
            logger.warning(f"알 수 없는 모델: {model}, gpt-4o-mini 가격 적용")
            model = "gpt-4o-mini"
        
        prices = self.TOKEN_PRICES[model]
        input_cost = input_tokens * prices["input"]
        output_cost = output_tokens * prices["output"]
        
        return round(input_cost + output_cost, 6)
    
    @log_function_call
    def track_usage(self, user_id: str, model: str, input_tokens: int, 
                   output_tokens: int, processing_time: float):
        """사용량 추적 및 저장"""
        total_tokens = input_tokens + output_tokens
        cost = self.calculate_cost(model, input_tokens, output_tokens)
        today = datetime.now().strftime('%Y-%m-%d')
        
        # 일일 사용량 업데이트
        if user_id not in self.daily_usage:
            self.daily_usage[user_id] = {}
        if today not in self.daily_usage[user_id]:
            self.daily_usage[user_id][today] = {
                "requests": 0, "tokens": 0, "cost": 0.0, "first_request": datetime.now()
            }
        
        # 통계 업데이트
        self.daily_usage[user_id][today]["requests"] += 1
        self.daily_usage[user_id][today]["tokens"] += total_tokens
        self.daily_usage[user_id][today]["cost"] += cost
        
        # 세션 통계 업데이트
        self.session_stats["total_requests"] += 1
        self.session_stats["total_tokens"] += total_tokens
        self.session_stats["total_cost"] += cost
        
        # 상세 로깅
        logger.info(
            f"토큰 사용량 추적 - 사용자: {user_id}, 모델: {model}, "
            f"입력: {input_tokens}, 출력: {output_tokens}, 총합: {total_tokens}, "
            f"비용: ${cost:.4f}, 처리시간: {processing_time:.2f}초"
        )
    
    def get_user_stats(self, user_id: str) -> Dict[str, Any]:
        """사용자 통계 조회"""
        today = datetime.now().strftime('%Y-%m-%d')
        
        if user_id in self.daily_usage and today in self.daily_usage[user_id]:
            stats = self.daily_usage[user_id][today]
            return {
                "date": today,
                "requests": stats["requests"],
                "tokens_used": stats["tokens"],
                "cost": round(stats["cost"], 4),
                "remaining_tokens": max(0, DEFAULT_CONFIG["daily_token_limit"] - stats["tokens"]),
                "limit_exceeded": stats["tokens"] >= DEFAULT_CONFIG["daily_token_limit"]
            }
        
        return {
            "date": today,
            "requests": 0,
            "tokens_used": 0,
            "cost": 0.0,
            "remaining_tokens": DEFAULT_CONFIG["daily_token_limit"],
            "limit_exceeded": False
        }

# 테스트
monitor = TokenMonitor()
print("✅ 토큰 모니터 생성 완료")

# 비용 계산 테스트
test_cost = monitor.calculate_cost("gpt-4o-mini", 100, 50)
print(f"💰 테스트 비용 계산 (gpt-4o-mini, 100+50 토큰): ${test_cost:.6f}")

## 5️⃣ API 키 로테이션 시스템

실무에서는 **안정성**을 위해 여러 API 키를 준비하고 로테이션하는 것이 좋습니다.

In [ ]:
class APIKeyRotator:
    """API 키 로테이션 및 실패 처리"""
    
    def __init__(self, primary_key: str, backup_key: Optional[str] = None):
        self.primary_key = primary_key
        self.backup_key = backup_key
        self.current_key = primary_key
        self.failure_count = 0
        self.last_rotation = datetime.now()
        self.max_failures = 3
        logger.info(f"API 키 로테이터 초기화 - 백업 키: {'있음' if backup_key else '없음'}")
    
    def get_current_key(self) -> str:
        """현재 사용할 API 키 반환"""
        return self.current_key
    
    def handle_api_error(self, error: Exception) -> bool:
        """
        API 에러 처리 및 키 로테이션
        
        Returns:
            bool: 로테이션 성공 여부 (재시도 가능)
        """
        self.failure_count += 1
        error_msg = str(error)
        
        logger.warning(f"API 호출 실패 ({self.failure_count}/{self.max_failures}): {error_msg}")
        
        # Rate limit 또는 반복 실패 시 백업 키로 전환
        if (self.failure_count >= self.max_failures and 
            self.backup_key and self.current_key != self.backup_key):
            
            logger.info("백업 API 키로 전환")
            self.current_key = self.backup_key
            self.failure_count = 0
            self.last_rotation = datetime.now()
            return True
        
        # 백업 키도 실패하면 쿨다운 후 primary로 복원
        if (self.current_key == self.backup_key and 
            self.failure_count >= self.max_failures):
            
            cooldown_period = timedelta(minutes=5)
            if datetime.now() - self.last_rotation > cooldown_period:
                logger.info("쿨다운 완료, Primary 키로 복원")
                self.current_key = self.primary_key
                self.failure_count = 0
                self.last_rotation = datetime.now()
                return True
        
        return False
    
    def reset_failure_count(self):
        """성공 시 실패 카운트 리셋"""
        if self.failure_count > 0:
            logger.debug("API 호출 성공, 실패 카운트 리셋")
            self.failure_count = 0

# API 키 로테이터 생성
key_rotator = APIKeyRotator(
    primary_key=DEFAULT_CONFIG["api_key"],
    backup_key=os.getenv("OPENAI_BACKUP_API_KEY")  # 선택사항
)

print("✅ API 키 로테이터 생성 완료")
print(f"🔑 현재 키: {key_rotator.get_current_key()[:8]}...{key_rotator.get_current_key()[-4:]}")

## 6️⃣ 기본 챗봇 클래스 구현

이제 실제 챗봇을 구현해봅시다. **일반 응답**과 **스트리밍 응답** 두 가지 방식을 지원합니다.

In [ ]:
class BasicChatbot:
    """기본 AI 챗봇 클래스"""
    
    def __init__(self, token_monitor: TokenMonitor, key_rotator: APIKeyRotator):
        self.token_monitor = token_monitor
        self.key_rotator = key_rotator
        self.client = None
        self._init_openai_client()
    
    def _init_openai_client(self):
        """OpenAI 클라이언트 초기화"""
        try:
            self.client = OpenAI(api_key=self.key_rotator.get_current_key())
            logger.info("OpenAI 클라이언트 초기화 완료")
        except Exception as e:
            logger.error(f"OpenAI 클라이언트 초기화 실패: {e}")
            raise
    
    def _refresh_client_if_needed(self):
        """키 로테이션 후 클라이언트 갱신"""
        current_key = self.key_rotator.get_current_key()
        if self.client.api_key != current_key:
            self.client.api_key = current_key
            logger.info("클라이언트 API 키 갱신")
    
    @log_function_call
    def generate_response(self, messages: List[Dict[str, str]], 
                         user_id: str = "anonymous") -> ChatMessage:
        """
        일반 응답 생성 (비스트리밍)
        
        Args:
            messages: 대화 메시지 리스트 [{'role': 'user', 'content': '...'}, ...]
            user_id: 사용자 ID (사용량 추적용)
            
        Returns:
            ChatMessage: 생성된 응답 메시지
        """
        start_time = time.time()
        
        # 사용량 제한 체크
        user_stats = self.token_monitor.get_user_stats(user_id)
        if user_stats["limit_exceeded"]:
            raise Exception(f"일일 토큰 한도 초과 ({user_stats['tokens_used']} 토큰)")
        
        try:
            self._refresh_client_if_needed()
            
            # OpenAI API 호출
            logger.info(f"API 요청 시작 - 모델: {DEFAULT_CONFIG['model']}, 메시지 수: {len(messages)}")
            
            response = self.client.chat.completions.create(
                model=DEFAULT_CONFIG["model"],
                messages=messages,
                max_tokens=DEFAULT_CONFIG["max_tokens"],
                temperature=DEFAULT_CONFIG["temperature"],
                stream=False
            )
            
            # 응답 데이터 처리
            processing_time = time.time() - start_time
            input_tokens = response.usage.prompt_tokens
            output_tokens = response.usage.completion_tokens
            content = response.choices[0].message.content
            
            # 사용량 추적
            self.token_monitor.track_usage(
                user_id, DEFAULT_CONFIG["model"], 
                input_tokens, output_tokens, processing_time
            )
            
            # 성공 시 실패 카운트 리셋
            self.key_rotator.reset_failure_count()
            
            # ChatMessage 객체 생성
            message = ChatMessage(
                role="assistant",
                content=content,
                timestamp=datetime.now(),
                tokens_used=response.usage.total_tokens,
                processing_time=processing_time,
                model=DEFAULT_CONFIG["model"]
            )
            
            logger.info(f"응답 생성 완료 - 토큰: {response.usage.total_tokens}, 시간: {processing_time:.2f}초")
            return message
            
        except Exception as e:
            # 에러 처리 및 키 로테이션
            if self.key_rotator.handle_api_error(e):
                logger.info("키 로테이션 후 재시도")
                return self.generate_response(messages, user_id)
            
            logger.error(f"응답 생성 실패: {e}")
            raise
    
    @log_function_call
    def generate_streaming_response(self, messages: List[Dict[str, str]], 
                                  user_id: str = "anonymous") -> Iterator[Dict[str, Any]]:
        """
        스트리밍 응답 생성
        
        Args:
            messages: 대화 메시지 리스트
            user_id: 사용자 ID
            
        Yields:
            Dict: 스트리밍 청크 데이터
                - type: "content" | "complete" | "error"
                - content: 텍스트 내용
                - finished: 완료 여부
        """
        start_time = time.time()
        accumulated_content = ""
        
        # 사용량 제한 체크
        user_stats = self.token_monitor.get_user_stats(user_id)
        if user_stats["limit_exceeded"]:
            yield {
                "type": "error",
                "content": f"일일 토큰 한도 초과 ({user_stats['tokens_used']} 토큰)",
                "finished": True
            }
            return
        
        try:
            self._refresh_client_if_needed()
            
            logger.info(f"스트리밍 응답 시작 - 사용자: {user_id}")
            
            # OpenAI 스트리밍 API 호출
            stream = self.client.chat.completions.create(
                model=DEFAULT_CONFIG["model"],
                messages=messages,
                max_tokens=DEFAULT_CONFIG["max_tokens"],
                temperature=DEFAULT_CONFIG["temperature"],
                stream=True
            )
            
            # 스트리밍 청크 처리
            chunk_count = 0
            for chunk in stream:
                if chunk.choices[0].delta.content:
                    content_chunk = chunk.choices[0].delta.content
                    accumulated_content += content_chunk
                    chunk_count += 1
                    
                    yield {
                        "type": "content",
                        "content": content_chunk,
                        "accumulated": accumulated_content,
                        "finished": False,
                        "chunk_number": chunk_count
                    }
            
            # 완료 처리
            processing_time = time.time() - start_time
            
            # 토큰 수 추정 (4 chars ≈ 1 token)
            estimated_tokens = len(accumulated_content) // 4
            estimated_input = len(str(messages)) // 4
            estimated_output = estimated_tokens
            
            # 사용량 추적 (추정값)
            self.token_monitor.track_usage(
                user_id, DEFAULT_CONFIG["model"],
                estimated_input, estimated_output, processing_time
            )
            
            self.key_rotator.reset_failure_count()
            
            yield {
                "type": "complete",
                "content": accumulated_content,
                "finished": True,
                "processing_time": processing_time,
                "estimated_tokens": estimated_tokens,
                "chunk_count": chunk_count
            }
            
            logger.info(f"스트리밍 완료 - 청크: {chunk_count}, 시간: {processing_time:.2f}초")
            
        except Exception as e:
            if self.key_rotator.handle_api_error(e):
                logger.info("키 로테이션 후 스트리밍 재시도")
                yield from self.generate_streaming_response(messages, user_id)
                return
            
            logger.error(f"스트리밍 응답 실패: {e}")
            yield {
                "type": "error",
                "content": f"오류가 발생했습니다: {str(e)}",
                "finished": True
            }

# 챗봇 인스턴스 생성
if DEFAULT_CONFIG["api_key"]:
    chatbot = BasicChatbot(monitor, key_rotator)
    print("✅ 챗봇 인스턴스 생성 완료")
else:
    print("❌ API 키가 없어 챗봇을 생성할 수 없습니다.")

## 7️⃣ 테스트 1: 일반 응답 생성

먼저 **비스트리밍** 방식으로 응답을 생성해봅시다.

In [ ]:
if 'chatbot' in locals():
    # 테스트 메시지
    test_messages = [
        {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다. 간결하고 친근하게 답변해주세요."},
        {"role": "user", "content": "안녕하세요! AI 챗봇에 대해 간단히 소개해주세요."}
    ]
    
    print("🚀 일반 응답 테스트 시작...")
    print("-" * 50)
    
    try:
        # 일반 응답 생성
        response = chatbot.generate_response(test_messages, "test_user")
        
        # 결과 출력
        print(f"🤖 응답: {response.content}")
        print(f"\n📊 메타데이터:")
        print(f"  - 모델: {response.model}")
        print(f"  - 토큰 사용량: {response.tokens_used}")
        print(f"  - 처리 시간: {response.processing_time:.2f}초")
        print(f"  - 생성 시각: {response.timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
        
        # 사용량 통계 확인
        user_stats = monitor.get_user_stats("test_user")
        print(f"\n📈 사용자 통계:")
        print(f"  - 오늘 요청 수: {user_stats['requests']}")
        print(f"  - 오늘 토큰 사용량: {user_stats['tokens_used']}")
        print(f"  - 오늘 예상 비용: ${user_stats['cost']:.4f}")
        print(f"  - 남은 토큰 한도: {user_stats['remaining_tokens']}")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
else:
    print("❌ 챗봇이 초기화되지 않았습니다. API 키를 확인해주세요.")

## 8️⃣ 테스트 2: 스트리밍 응답 생성

이번에는 **스트리밍** 방식으로 실시간 응답을 확인해봅시다.

In [ ]:
if 'chatbot' in locals():
    # 스트리밍 테스트 메시지
    streaming_messages = [
        {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다. 상세하고 친근하게 답변해주세요."},
        {"role": "user", "content": "Python으로 ChatGPT와 같은 챗봇을 만드는 방법을 단계별로 설명해주세요."}
    ]
    
    print("🌊 스트리밍 응답 테스트 시작...")
    print("-" * 50)
    print("🤖 AI (실시간): ", end="", flush=True)
    
    try:
        full_response = ""
        chunk_count = 0
        
        # 스트리밍 응답 처리
        for chunk in chatbot.generate_streaming_response(streaming_messages, "streaming_test_user"):
            if chunk["type"] == "content":
                print(chunk["content"], end="", flush=True)
                full_response += chunk["content"]
                chunk_count += 1
            
            elif chunk["type"] == "complete":
                print(f"\n\n📊 스트리밍 완료:")
                print(f"  - 총 청크 수: {chunk['chunk_count']}")
                print(f"  - 처리 시간: {chunk['processing_time']:.2f}초")
                print(f"  - 추정 토큰: {chunk['estimated_tokens']}")
                print(f"  - 응답 길이: {len(full_response)} 문자")
                
                # 청크당 평균 처리 시간
                avg_chunk_time = chunk['processing_time'] / max(chunk['chunk_count'], 1)
                print(f"  - 평균 청크 시간: {avg_chunk_time:.3f}초")
            
            elif chunk["type"] == "error":
                print(f"\n❌ 오류: {chunk['content']}")
                break
        
        # 스트리밍 사용자 통계 확인
        streaming_stats = monitor.get_user_stats("streaming_test_user")
        print(f"\n📈 스트리밍 사용자 통계:")
        print(f"  - 요청 수: {streaming_stats['requests']}")
        print(f"  - 토큰 사용량: {streaming_stats['tokens_used']} (추정)")
        print(f"  - 예상 비용: ${streaming_stats['cost']:.4f}")
        
    except Exception as e:
        print(f"\n❌ 스트리밍 오류: {e}")
else:
    print("❌ 챗봇이 초기화되지 않았습니다.")

## 9️⃣ 테스트 3: 대화형 인터랙션

실제 대화처럼 **연속적인 메시지 교환**을 테스트해봅시다.

In [ ]:
if 'chatbot' in locals():
    print("💬 대화형 테스트 시작")
    print("=" * 60)
    
    # 대화 시뮬레이션
    conversation = [
        {"role": "system", "content": "당신은 Python 프로그래밍 전문가입니다. 실무 경험을 바탕으로 답변해주세요."},
    ]
    
    test_questions = [
        "Python에서 비동기 프로그래밍이란 무엇인가요?",
        "async/await와 threading의 차이점을 설명해주세요.",
        "실무에서 비동기를 언제 사용하면 좋을까요?"
    ]
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n👤 질문 {i}: {question}")
        conversation.append({"role": "user", "content": question})
        
        print(f"🤖 답변 {i}: ", end="", flush=True)
        
        try:
            # 스트리밍으로 응답 생성
            response_text = ""
            for chunk in chatbot.generate_streaming_response(conversation, f"conversation_test_{i}"):
                if chunk["type"] == "content":
                    print(chunk["content"], end="", flush=True)
                    response_text += chunk["content"]
                elif chunk["type"] == "complete":
                    print(f"\n   ⏱️  {chunk['processing_time']:.2f}초 | 📊 ~{chunk['estimated_tokens']} 토큰")
                elif chunk["type"] == "error":
                    print(f"\n   ❌ {chunk['content']}")
                    break
            
            # 대화에 AI 응답 추가
            if response_text:
                conversation.append({"role": "assistant", "content": response_text})
        
        except Exception as e:
            print(f"\n   ❌ 오류: {e}")
        
        print("-" * 40)
    
    # 전체 대화 통계
    print(f"\n📊 전체 세션 통계:")
    session_stats = monitor.session_stats
    session_duration = (datetime.now() - session_stats['start_time']).total_seconds()
    print(f"  - 총 요청 수: {session_stats['total_requests']}")
    print(f"  - 총 토큰 수: {session_stats['total_tokens']}")
    print(f"  - 총 비용: ${session_stats['total_cost']:.4f}")
    print(f"  - 세션 시간: {session_duration:.1f}초")
    print(f"  - 평균 응답 시간: {session_duration / max(session_stats['total_requests'], 1):.2f}초")
else:
    print("❌ 챗봇이 초기화되지 않았습니다.")

## 🔟 테스트 4: 에러 처리 및 한도 제한

실무에서는 **에러 상황**과 **사용량 제한**을 적절히 처리해야 합니다.

In [ ]:
if 'chatbot' in locals():
    print("🧪 에러 처리 및 한도 제한 테스트")
    print("=" * 50)
    
    # 1. 사용량 한도 초과 시뮬레이션
    print("\n1️⃣ 사용량 한도 초과 테스트")
    
    # 임시로 한도를 낮게 설정
    original_limit = DEFAULT_CONFIG["daily_token_limit"]
    DEFAULT_CONFIG["daily_token_limit"] = 10  # 매우 낮은 한도
    
    # 사용량을 인위적으로 증가
    monitor.track_usage("limit_test_user", "gpt-4o-mini", 5, 10, 0.5)
    
    try:
        response = chatbot.generate_response(
            [{"role": "user", "content": "안녕하세요"}], 
            "limit_test_user"
        )
        print("❌ 예상과 다름: 한도 초과 에러가 발생해야 함")
    except Exception as e:
        print(f"✅ 예상된 한도 초과 에러: {e}")
    
    # 한도 복원
    DEFAULT_CONFIG["daily_token_limit"] = original_limit
    
    # 2. 잘못된 API 키 테스트 (백업이 없는 경우)
    print("\n2️⃣ 잘못된 API 키 처리 테스트")
    
    # 임시로 잘못된 키 설정
    original_key = key_rotator.primary_key
    key_rotator.primary_key = "sk-invalid-key-for-testing"
    key_rotator.current_key = "sk-invalid-key-for-testing"
    
    # 새 챗봇 인스턴스로 테스트
    test_rotator = APIKeyRotator("sk-invalid-key", None)
    test_chatbot = BasicChatbot(TokenMonitor(), test_rotator)
    
    try:
        response = test_chatbot.generate_response(
            [{"role": "user", "content": "테스트"}], 
            "error_test_user"
        )
        print("❌ 예상과 다름: API 키 에러가 발생해야 함")
    except Exception as e:
        print(f"✅ 예상된 API 키 에러: {type(e).__name__}")
    
    # 원래 키 복원
    key_rotator.primary_key = original_key
    key_rotator.current_key = original_key
    
    print("\n✅ 에러 처리 테스트 완료")
else:
    print("❌ 챗봇이 초기화되지 않았습니다.")

## 1️⃣1️⃣ 실무 팁: 성능 비교 분석

**스트리밍 vs 일반 응답**의 차이점을 실제로 측정해봅시다.

In [ ]:
if 'chatbot' in locals():
    print("⚡ 성능 비교 분석")
    print("=" * 50)
    
    test_prompt = "딥러닝과 머신러닝의 차이점을 자세히 설명해주세요."
    messages = [
        {"role": "system", "content": "당신은 AI/ML 전문가입니다."},
        {"role": "user", "content": test_prompt}
    ]
    
    results = {}
    
    # 1. 일반 응답 측정
    print("\n📊 일반 응답 모드 측정 중...")
    try:
        start_time = time.time()
        normal_response = chatbot.generate_response(messages, "perf_test_normal")
        
        results["normal"] = {
            "total_time": normal_response.processing_time,
            "tokens": normal_response.tokens_used,
            "response_length": len(normal_response.content),
            "first_token_time": normal_response.processing_time,  # 전체 응답 완료 시간
        }
        
        print(f"  ✅ 완료 - {normal_response.processing_time:.2f}초, {normal_response.tokens_used} 토큰")
    
    except Exception as e:
        print(f"  ❌ 실패: {e}")
        results["normal"] = None
    
    # 2. 스트리밍 응답 측정
    print("\n🌊 스트리밍 응답 모드 측정 중...")
    try:
        streaming_start = time.time()
        first_chunk_time = None
        chunk_times = []
        total_chunks = 0
        accumulated_text = ""
        
        for chunk in chatbot.generate_streaming_response(messages, "perf_test_streaming"):
            current_time = time.time()
            
            if chunk["type"] == "content":
                if first_chunk_time is None:
                    first_chunk_time = current_time - streaming_start
                
                chunk_times.append(current_time - streaming_start)
                accumulated_text += chunk["content"]
                total_chunks += 1
                
                # 진행률 표시 (매 10청크마다)
                if total_chunks % 10 == 0:
                    print(f"  📊 청크 {total_chunks}: {len(accumulated_text)} 문자 누적")
            
            elif chunk["type"] == "complete":
                results["streaming"] = {
                    "total_time": chunk["processing_time"],
                    "tokens": chunk["estimated_tokens"],
                    "response_length": len(accumulated_text),
                    "first_token_time": first_chunk_time,
                    "total_chunks": total_chunks,
                    "avg_chunk_interval": chunk["processing_time"] / max(total_chunks, 1)
                }
                print(f"  ✅ 완료 - {chunk['processing_time']:.2f}초, {total_chunks} 청크")
                break
            
            elif chunk["type"] == "error":
                print(f"  ❌ 에러: {chunk['content']}")
                results["streaming"] = None
                break
    
    except Exception as e:
        print(f"  ❌ 스트리밍 실패: {e}")
        results["streaming"] = None
    
    # 3. 결과 비교 분석
    print("\n📈 성능 비교 결과")
    print("=" * 50)
    
    if results["normal"] and results["streaming"]:
        normal = results["normal"]
        streaming = results["streaming"]
        
        print(f"📊 **처리 시간 비교**")
        print(f"  일반 모드: {normal['total_time']:.2f}초 (전체 응답 완료까지)")
        print(f"  스트리밍: {streaming['total_time']:.2f}초 (전체 응답 완료까지)")
        print(f"  첫 토큰: {streaming['first_token_time']:.2f}초 (사용자가 응답을 보기 시작)")
        
        # 사용자 경험 개선 계산
        ux_improvement = ((normal['total_time'] - streaming['first_token_time']) / normal['total_time']) * 100
        print(f"  🚀 UX 개선: {ux_improvement:.1f}% (첫 응답까지 시간 단축)")
        
        print(f"\n📊 **응답 품질 비교**")
        print(f"  일반 모드: {normal['response_length']} 문자, {normal['tokens']} 토큰")
        print(f"  스트리밍: {streaming['response_length']} 문자, ~{streaming['tokens']} 토큰 (추정)")
        
        print(f"\n⚡ **스트리밍 세부 정보**")
        print(f"  총 청크 수: {streaming['total_chunks']}")
        print(f"  평균 청크 간격: {streaming['avg_chunk_interval']:.3f}초")
        print(f"  초당 문자 수: {streaming['response_length'] / streaming['total_time']:.1f} chars/sec")
    
    else:
        print("❌ 비교할 수 있는 데이터가 부족합니다.")
else:
    print("❌ 챗봇이 초기화되지 않았습니다.")

## 1️⃣2️⃣ 로그 분석 및 디버깅

생성된 **로그 파일**을 분석하여 시스템 동작을 확인해봅시다.

In [ ]:
# 로그 파일 분석
log_file = "chatbot_lesson1.log"

if os.path.exists(log_file):
    print(f"📋 로그 파일 분석: {log_file}")
    print("=" * 50)
    
    with open(log_file, 'r', encoding='utf-8') as f:
        log_lines = f.readlines()
    
    # 로그 통계
    stats = {
        "총 로그 라인": len(log_lines),
        "INFO 로그": sum(1 for line in log_lines if " - INFO - " in line),
        "DEBUG 로그": sum(1 for line in log_lines if " - DEBUG - " in line),
        "WARNING 로그": sum(1 for line in log_lines if " - WARNING - " in line),
        "ERROR 로그": sum(1 for line in log_lines if " - ERROR - " in line),
    }
    
    print("📊 로그 통계:")
    for key, value in stats.items():
        print(f"  {key}: {value}")
    
    # 최근 로그 표시 (마지막 10줄)
    print("\n📄 최근 로그 (마지막 10줄):")
    print("-" * 30)
    for line in log_lines[-10:]:
        print(f"  {line.strip()}")
    
    # API 호출 관련 로그 필터링
    api_logs = [line for line in log_lines if "API" in line or "토큰" in line]
    if api_logs:
        print(f"\n🔍 API 관련 로그 ({len(api_logs)}개):")
        print("-" * 30)
        for log in api_logs[-5:]:  # 최근 5개만
            print(f"  {log.strip()}")
else:
    print(f"❌ 로그 파일을 찾을 수 없습니다: {log_file}")

## 1️⃣3️⃣ 종합 데모 및 결과 정리

지금까지 구현한 기능들을 종합하여 **최종 데모**를 실행해봅시다.

In [ ]:
print("🎉 1차시 종합 데모")
print("=" * 60)

if 'chatbot' in locals():
    # 최종 통계 수집
    session_stats = monitor.session_stats
    session_duration = (datetime.now() - session_stats['start_time']).total_seconds()
    
    print(f"📊 **전체 세션 요약**")
    print(f"  - 세션 시간: {session_duration:.1f}초")
    print(f"  - 총 API 요청: {session_stats['total_requests']}회")
    print(f"  - 총 토큰 사용: {session_stats['total_tokens']}개")
    print(f"  - 총 예상 비용: ${session_stats['total_cost']:.4f}")
    
    if session_stats['total_requests'] > 0:
        avg_tokens = session_stats['total_tokens'] / session_stats['total_requests']
        avg_cost = session_stats['total_cost'] / session_stats['total_requests']
        print(f"  - 평균 토큰/요청: {avg_tokens:.1f}개")
        print(f"  - 평균 비용/요청: ${avg_cost:.4f}")
    
    print(f"\n🎯 **1차시 핵심 학습 내용**")
    print(f"  ✅ OpenAI API 연동 및 클라이언트 설정")
    print(f"  ✅ 스트리밍 vs 일반 응답 구현")
    print(f"  ✅ 토큰 사용량 모니터링 및 비용 계산")
    print(f"  ✅ API 키 로테이션 및 에러 처리")
    print(f"  ✅ 구조화된 로깅 시스템")
    print(f"  ✅ 사용량 제한 및 보안 고려사항")
    
    print(f"\n🚀 **다음 차시 미리보기**")
    print(f"  - 2차시: 프롬프트 엔지니어링 및 템플릿 시스템")
    print(f"  - 3차시: RAG 구현 (문서 검색 기반 답변)")
    print(f"  - 4차시: 대화 상태 관리 및 멀티턴 최적화")
    
    print(f"\n💡 **실무 적용 팁**")
    print(f"  - 프로덕션에서는 Redis 클러스터 사용 권장")
    print(f"  - API 키는 환경변수나 비밀 관리 서비스 사용")
    print(f"  - 로그는 ELK 스택이나 클라우드 로깅 서비스 연동")
    print(f"  - 사용량 알림 시스템 구축 (Slack, 이메일)")
    print(f"  - A/B 테스트를 위한 모델별 응답 비교")

else:
    print("❌ 챗봇이 초기화되지 않았습니다.")

print("\n🎊 1차시 실습 완료! 수고하셨습니다!")

## 📝 숙제 및 확장 과제

### 🏠 숙제
1. **다른 LLM 모델 테스트**: GPT-4, Claude, Gemini 등 비교 분석
2. **사용량 알림 시스템**: 일정 토큰 수 초과 시 이메일/Slack 알림
3. **응답 품질 평가**: 같은 질문에 대한 모델별 응답 점수화

### 🚀 확장 과제
1. **멀티 모델 지원**: 질문 유형에 따라 최적 모델 자동 선택
2. **캐싱 시스템**: 동일 질문 빠른 응답 (Redis)
3. **대화 히스토리**: 이전 대화 맥락 유지
4. **A/B 테스트**: 프롬프트 변경에 따른 응답 품질 측정

### 📚 추천 자료
- [OpenAI API 공식 문서](https://platform.openai.com/docs)
- [Streamlit 공식 튜토리얼](https://docs.streamlit.io)
- [Python 로깅 베스트 프랙티스](https://docs.python.org/3/howto/logging.html)

---

**다음 시간에는 프롬프트 최적화와 템플릿 시스템을 배워봅시다! 🎯**